In [ ]:
#stable diffusion part
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate

In [ ]:
#huggingface token must be entered
!huggingface-cli login

In [3]:
#imports
import json
import os
import shutil
import re
from diffusers import StableDiffusionPipeline
import mediapy as media
import torch
from torch import autocast
from IPython.display import Image

In [ ]:
from google.colab import drive
import os

#go to drive
drive.mount('/content/drive')

# set root folder path as stablediffautomation folder inside MyDrive
root_folder = '/content/drive/MyDrive/stablediffautomation'

# path to the data folder inside root folder
data_folder = os.path.join(root_folder, 'data')

In [5]:
def splitValseData(data_folder):

  #caption_pattern = r'"caption":\s*"(([^"]*))"'
  #foil_pattern = r'"foil":\s*"(([^"]*))"'

  dict_captions_all={}   
  dict_foils_all={}
  

  # get the list of JSON files in the data folder
  json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]


  for json_file in json_files:
      # create a folder to hold stable diff images according to the classes
      folder_name = json_file[:-5]  # remove the '.json' extension
      folder_path = os.path.join(data_folder, folder_name)
      os.makedirs(folder_path, exist_ok=True)


      dict_captions_all[folder_name]=[] 
      dict_foils_all[folder_name]=[] 
      
      json_path = os.path.join(data_folder, json_file)
      with open(json_path) as f:
        data = json.load(f)

        #split caption and foils of each class
        for i in data:
          dict_captions_all[folder_name].append(data[i]["caption"])
          dict_foils_all[folder_name].append(data[i]["foil"])
  
  return dict_captions_all, dict_foils_all


dict_captions_all, dict_foils_all = splitValseData(data_folder)

In [6]:
def stableDiffImg(file_path,prompt_sentence):

  model_id = "runwayml/stable-diffusion-v1-5"
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
  pipe = pipe.to("cuda")

  prompt = prompt_sentence
  image = pipe(prompt).images[0]  

  # create directory if it doesn't exist
  os.makedirs(os.path.dirname(file_path), exist_ok=True)

  image.save(file_path)
  print("Image saved in: "+str(file_path))


In [ ]:
#generate image for all captions of each class
for i in dict_captions_all:
  for j in dict_captions_all[i]:
    file_path = os.path.join(data_folder+"/"+i, "caption", j + '.png')
    #if image exist dont generate again, if you want multiple images of the same caption hold a counter etc.
    if os.path.exists(file_path):
      continue
    stableDiffImg(file_path,j)

#generate image for all foils of each class
for i in dict_foils_all:
  for j in dict_foils_all[i]:
    file_path = os.path.join(data_folder+"/"+i, "foil", j + '.png')
    if os.path.exists(file_path):
      continue
    stableDiffImg(file_path,j)